In [2]:
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.models import NBEATSModel

def train_model(model_args, callbacks, train, val):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    # Create the model using model_args from Ray Tune
    model = NBEATSModel(
        input_chunk_length=24,
        output_chunk_length=12,
        n_epochs=500,
        torch_metrics=torch_metrics,
        pl_trainer_kwargs={"callbacks": callbacks, "enable_progress_bar": False},
        **model_args)

    model.fit(
        series=train,
        val_series=val,
    )

# Read data:
series = AirPassengersDataset().load()

# Create training and validation sets:
train, val = series.split_after(pd.Timestamp(year=1957, month=12, day=1))

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
transformer.fit(train)
train = transformer.transform(train)
val = transformer.transform(val)

# Early stop callback
my_stopper = EarlyStopping(
    monitor="val_MeanAbsolutePercentageError",
    patience=5,
    min_delta=0.05,
    mode='min',
)

# set up ray tune callback
tune_callback = TuneReportCallback(
    {
        "loss": "val_Loss",
        "MAPE": "val_MeanAbsolutePercentageError",
    },
    on="validation_end",
)

# define the hyperparameter space
config = {
    "batch_size": tune.choice([16, 32, 64, 128]),
    "num_blocks": tune.choice([1, 2, 3, 4, 5]),
    "num_stacks": tune.choice([32, 64, 128]),
    "dropout": tune.uniform(0, 0.2),
}

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

resources_per_trial = {"cpu": 16, "gpu": 1}

# the number of combinations to try
num_samples = 10

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train, val=val,
)

# optimize hyperparameters by minimizing the MAPE on the validation set
analysis = tune.run(
    train_fn_with_parameters,
    resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="MAPE",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

== Status ==
Current time: 2023-01-30 19:06:29 (running for 00:00:00.20)
Memory usage on this node: 16.0/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/10.11 GiB heap, 0.0/5.06 GiB objects
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+--------------+------------+
| Trial name              | status   | loc             |   batch_size |   num_blocks |   num_stacks |    dropout |
|-------------------------+----------+-----------------+--------------+--------------+--------------+------------|
| train_model_25b11_00000 | RUNNING  | 127.0.0.1:29528 |          128 |            3 |           32 | 0.021416   |
| train_model

(train_model pid=29528) GPU available: True (cuda), used: False
(train_model pid=29528) TPU available: False, using: 0 TPU cores
(train_model pid=29528) IPU available: False, using: 0 IPUs
(train_model pid=29528) HPU available: False, using: 0 HPUs
(train_model pid=29528) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=29528)   rank_zero_warn(
(train_model pid=29528) 
(train_model pid=29528)   | Name          | Type             | Params
(train_model pid=29528) ---------------------------------------------------
(train_model pid=29528) 0 | criterion     | MSELoss          | 0     
(train_model pid=29528) 1 | train_metrics | MetricCollection | 0     
(train_model pid=29528) 2 | val_metrics   | MetricCollection | 0     
(train_model pid=29528) 3 | stacks        | ModuleList       | 19.8 M
(train_mode

Trial name,MAPE,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_25b11_00000,0.135828,2023-01-30_19-06-47,True,,16dc52cfafb340f6b61c066d3fa31aa5,Tensortorch,19,127.0.0.1,29528,13.0008,0.654763,13.0008,1675076807,0,,19,25b11_00000,0.00414515
train_model_25b11_00001,1.32436,2023-01-30_19-07-25,True,,d76c7e79ecd5404e922023eed73f9778,Tensortorch,3,127.0.0.1,24532,31.9876,10.0011,31.9876,1675076845,0,,3,25b11_00001,0.00511074
train_model_25b11_00002,0.731292,2023-01-30_19-07-33,True,,71508ac2f2a64c8e933d9335c0428cf3,Tensortorch,6,127.0.0.1,5548,3.04695,0.454168,3.04695,1675076853,0,,6,25b11_00002,0.00501823
train_model_25b11_00003,0.148242,2023-01-30_19-07-46,True,,eedebf5888f04aec929486765f6fffa0,Tensortorch,16,127.0.0.1,1812,7.3136,0.409638,7.3136,1675076866,0,,16,25b11_00003,0.0045054
train_model_25b11_00004,0.396265,2023-01-30_19-07-56,True,,1a0910ec2ce44660ac64076873a349d2,Tensortorch,6,127.0.0.1,30116,4.17662,0.630519,4.17662,1675076876,0,,6,25b11_00004,0.00511813
train_model_25b11_00005,0.141592,2023-01-30_19-08-11,True,,25f5ad96e9e0443184d560857d6bcd7f,Tensortorch,14,127.0.0.1,8832,8.69273,0.594927,8.69273,1675076891,0,,14,25b11_00005,0.00357771
train_model_25b11_00006,0.749197,2023-01-30_19-08-18,True,,36ae72ae14784be18ccccdae85631393,Tensortorch,3,127.0.0.1,6084,1.64744,0.431654,1.64744,1675076898,0,,3,25b11_00006,0.00292468
train_model_25b11_00007,0.162758,2023-01-30_19-08-40,True,,b6e854fd6e014008ad19c357105d89c9,Tensortorch,10,127.0.0.1,15980,17.3867,1.69241,17.3867,1675076920,0,,10,25b11_00007,0.00447035
train_model_25b11_00008,0.52124,2023-01-30_19-08-51,True,,34f0eca718b246debbb4447ced8659b3,Tensortorch,6,127.0.0.1,11172,4.39312,0.656506,4.39312,1675076931,0,,6,25b11_00008,0.00453234
train_model_25b11_00009,0.9174,2023-01-30_19-09-03,True,,87f8e885013d43589bdc3044a0c075c7,Tensortorch,3,127.0.0.1,29712,6.40906,1.83459,6.40906,1675076943,0,,3,25b11_00009,0.00500107


== Status ==
Current time: 2023-01-30 19:06:35 (running for 00:00:06.18)
Memory usage on this node: 17.1/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/10.11 GiB heap, 0.0/5.06 GiB objects
Current best trial: 25b11_00000 with MAPE=0.9014418882324087 and parameters={'batch_size': 128, 'num_blocks': 3, 'num_stacks': 32, 'dropout': 0.021416047999032496}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+--------------+------------+----------+----------------------+
| Trial name              | status   | loc             |   batch_size |   num_blocks |   num_stacks |    dropout |     MAPE |   training_iteration |
|----------

(train_model pid=29528) 2023-01-30 19:06:35,825	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:36,485	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:37,131	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:37,772	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:38,423	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:39,076	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29528) 2023-01-30 19:06:39,749	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.cal

...

== Status ==
Current time: 2023-01-30 19:08:51 (running for 00:02:22.16)
Memory usage on this node: 15.8/31.6 GiB 
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.14830844367209076 | Iter 6.000: -0.3542216255122099 | Iter 3.000: -0.5750834949737773
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/10.11 GiB heap, 0.0/5.06 GiB objects
Current best trial: 25b11_00000 with MAPE=0.13582769678058468 and parameters={'batch_size': 128, 'num_blocks': 3, 'num_stacks': 32, 'dropout': 0.021416047999032496}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+--------------+------------+----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   num_blocks |   num_stacks |    d

(train_model pid=29712) GPU available: True (cuda), used: False
(train_model pid=29712) TPU available: False, using: 0 TPU cores
(train_model pid=29712) IPU available: False, using: 0 IPUs
(train_model pid=29712) HPU available: False, using: 0 HPUs
(train_model pid=29712) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=29712)   rank_zero_warn(
(train_model pid=29712) 
(train_model pid=29712)   | Name          | Type             | Params
(train_model pid=29712) ---------------------------------------------------
(train_model pid=29712) 0 | criterion     | MSELoss          | 0     
(train_model pid=29712) 1 | train_metrics | MetricCollection | 0     
(train_model pid=29712) 2 | val_metrics   | MetricCollection | 0     
(train_model pid=29712) 3 | stacks        | ModuleList       | 52.9 M
(train_mode

== Status ==
Current time: 2023-01-30 19:08:59 (running for 00:02:29.88)
Memory usage on this node: 17.9/31.6 GiB 
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.14830844367209076 | Iter 6.000: -0.3542216255122099 | Iter 3.000: -0.5750834949737773
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/10.11 GiB heap, 0.0/5.06 GiB objects
Current best trial: 25b11_00000 with MAPE=0.13582769678058468 and parameters={'batch_size': 128, 'num_blocks': 3, 'num_stacks': 32, 'dropout': 0.021416047999032496}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+--------------+------------+----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   num_blocks |   num_stacks |    d

(train_model pid=29712) 2023-01-30 19:08:59,498	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29712) 2023-01-30 19:09:01,299	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=29712) 2023-01-30 19:09:03,135	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
2023-01-30 19:09:03,304	INFO tune.py:762 -- Total run time: 153.65 seconds (153.50 seconds for the tuning loop).


== Status ==
Current time: 2023-01-30 19:09:03 (running for 00:02:33.52)
Memory usage on this node: 17.9/31.6 GiB 
Using AsyncHyperBand: num_stopped=6
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.14830844367209076 | Iter 6.000: -0.3542216255122099 | Iter 3.000: -0.5793413854557841
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/10.11 GiB heap, 0.0/5.06 GiB objects
Current best trial: 25b11_00000 with MAPE=0.13582769678058468 and parameters={'batch_size': 128, 'num_blocks': 3, 'num_stacks': 32, 'dropout': 0.021416047999032496}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+--------------+------------+----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   num_blocks |   num_stacks |    dropout |     MA